<a href="https://colab.research.google.com/github/vifirsanova/OCR_Glossary/blob/main/workbook_collocations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### ноутбук будет дополнен

# Импорт библиотек

In [2]:
# использование цвета для вывода на экран
!pip install colorama

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from pathlib import Path
import re
from colorama import Fore, Style
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize 
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import spacy
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import Phrases
from gensim.models.phrases import Phraser
import numpy as np
from gensim.corpora import Dictionary
import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


# Загрузка нескольких файлов

* загружаем файлы из GitHub
* соединяем их в одной переменной

In [4]:
!wget https://raw.githubusercontent.com/vifirsanova/OCR_Glossary/main/Geology.txt
!wget https://raw.githubusercontent.com/vifirsanova/OCR_Glossary/main/Physical_Geology_2nd_Ed.txt
!wget https://raw.githubusercontent.com/vifirsanova/OCR_Glossary/main/Lumen-Learning-Geology.txt

--2023-02-01 14:22:09--  https://raw.githubusercontent.com/vifirsanova/OCR_Glossary/main/Geology.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 207953 (203K) [text/plain]
Saving to: ‘Geology.txt.2’

Geology.txt.2       100%[===================>] 203.08K  --.-KB/s    in 0.03s   

2023-02-01 14:22:09 (5.85 MB/s) - ‘Geology.txt.2’ saved [207953/207953]

--2023-02-01 14:22:10--  https://raw.githubusercontent.com/vifirsanova/OCR_Glossary/main/Physical_Geology_2nd_Ed.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1268676 (1.2M

In [5]:
# проверяем наличие загруженных файлов в директории
# с помощью консольной команды ls

!ls

Geology.txt		      Lumen-Learning-Geology.txt.2
Geology.txt.1		      Physical_Geology_2nd_Ed.txt
Geology.txt.2		      Physical_Geology_2nd_Ed.txt.1
Lumen-Learning-Geology.txt    Physical_Geology_2nd_Ed.txt.2
Lumen-Learning-Geology.txt.1  sample_data


In [6]:
# прочитываем все файлы с помощью метода read_text из библиотеки Path
# мы можем выполнить это параллельно, 
# то есть за одну команду одновременно обработать несколько файлов

text0, text1, text2 = Path('Geology.txt').read_text(), Path('Lumen-Learning-Geology.txt').read_text(), Path('Physical_Geology_2nd_Ed.txt').read_text()

# посмотрим на часть одного из текстов

text2[:900]

'https://opentextbc.ca/physicalgeology2ed/\nPhysical Geology - 2nd Edition\nPhysical Geology - 2nd Edition\nSteven Earle\nKarla Panchuk\nBCcampus\nVictoria, B.C.\nIcon for the Creative Commons Attribution 4.0 International License\n\nPhysical Geology - 2nd Edition by Steven Earle is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted.\n\n© 2019 Steven Earle\n\nThe CC licence permits you to retain, reuse, copy, redistribute, and revise this book—in whole or in part—for free providing the author is attributed as follows:\n\nPhysical Geology – 2nd Edition by Steven Earle is used under a Creative Commons Attribution 4.0 International Licence.\nIf you redistribute all or part of this book, it is recommended the following statement be added to the copyright page so readers can access the original book at no cost:\n\nDownload for free from the B.C. Open Collection'

In [7]:
# сохраним этот отрывок в отдельную переменную для будущих экспериментов c RegEx

test = text2[:900]

Какие артефакты бросаются нам в глаза в этом тексте?

In [8]:
# объединим все файлы в один и посмотрим его статистику

text = " ".join([text0, text1, text2])

print(f'Статистическое описание сырого документа\n\n{Fore.RED}Длина документа:{Style.RESET_ALL} {len(text)} символов\n\
{Fore.RED}Приблизительное количество слов в документе:{Style.RESET_ALL} {len(text.split(" "))}')

Статистическое описание сырого документа

Длина документа: 2452019 символов
Приблизительное количество слов в документе: 375467


Какие статистические данные по сырому тексту нам могут быть интересны?

Как их получить?

# Предобработка данных

## Чистка данных с помощью RegEx

Вспомним отрывок одного из текстов:

```
https://opentextbc.ca/physicalgeology2ed/\nPhysical Geology - 2nd Edition\nPhysical Geology - 2nd Edition\nSteven Earle\nKarla Panchuk\nBCcampus\nVictoria, B.C.\nIcon for the Creative Commons Attribution 4.0 International License\n\nPhysical Geology - 2nd Edition by Steven Earle is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted.\n\n© 2019 Steven Earle\n\nThe CC licence permits you to retain, reuse, copy, redistribute, and revise this book—in whole or in part—for free providing the author is attributed as follows:\n\nPhysical Geology – 2nd Edition by Steven Earle is used under a Creative Commons Attribution 4.0 International Licence.\nIf you redistribute all or part of this book, it is recommended the following statement be added to the copyright page so readers can access the original book at no cost:\n\nDownload for free from the B.C. Open Collection

```

Мы обратили внимание на некоторые артефакты:

* ссылки
* абзацные отступы
* специальные символы, например `©`
* ???

Проверим, верна ли наша гипотеза.




In [9]:
# ссылки

link_pattern = re.compile(r'https?://\S+|www\.\S+')
links = re.findall(link_pattern, text)

print(f'{Fore.RED}Количество обнаруженных ссылок:{Style.RESET_ALL} {len(links)}\n{Fore.RED}Образцы ссылок:{Style.RESET_ALL} {links[:3]}')

# специальные символы

symbol_pattern = re.compile(r'[^a-zA-Z0-9 .,:;…()!?\-—]+')
symbols = re.findall(symbol_pattern, text)

print(f'\n{Fore.RED}Количество обнаруженных специальных символов:{Style.RESET_ALL} {len(symbols)}\n{Fore.RED}Образцы уникальных символов:{Style.RESET_ALL} {set(symbols[500:1000])}')

# любые символы

any_symbol_pattern = re.compile(r'[^a-zA-Z ]+')
all_symbols = re.findall(any_symbol_pattern, text)

print(f'\n{Fore.RED}Количество обнаруженных символов:{Style.RESET_ALL} {len(all_symbols)}\n{Fore.RED}Образцы уникальных символов:{Style.RESET_ALL} {set(all_symbols[500:600])}')

Количество обнаруженных ссылок: 341
Образцы ссылок: ['https://opengeology.org/textbook/1-understanding-science/', 'https://www.columbiabasin.edu/.', 'http://www.ck12.org/book/CK-12-Earth-Science-For-Middle-School/section/1.2/.']

Количество обнаруженных специальных символов: 23382
Образцы уникальных символов: {'\n', '–', '%', '•', '”', "'", '\n\n\n\n\n', 'ʻ', '°/', 'í', '\n\n\n', '+', '“', 'ᐧ', '>', '<', '’', '‘', 'ö', '·', '\n\n\n\n', ']', '°', 'ʻā', '/', '[', '\n\n\n\n\n\n', '&'}

Количество обнаруженных символов: 76614
Образцы уникальных символов: {'—', ':', ',', '-', '\n', '2.\n', '’', ').\n', '(', ';', '.\n1.5.2', '.', '),', '.\n1.5.3', '.\n'}


Мы хотим проверить, какое влияние окажет удаление знаков препинания на результат работы нашей программы.

Для этого создадим три функции для чистки текста: 
- первая сохраняет знаки препинания и цифры
- вторая очищает текст от знаков препинания
- третья удаляет все символы кроме букв

In [10]:
def clean(text):
  ### сохраняет знаки препинания и цифры ###

  # удаляем из текста ссылки 
  text = re.sub('https?://\S+|www\.\S+', ' ', text)
  
  # удаляем специальные символы
  # оставляем знаки препинания и цифры
  text = re.sub('[^a-zA-Z0-9 .,:;…()!?\-—]+', ' ', text)

  # возвращаем текст, очищенный от лишних пробелов
  # которые появились в результате обработки
  return re.sub('\s+', ' ', text)

def full_clean(text):
  ### очищает текст от знаков препинания ###

  # удаляем из текста ссылки 
  text = re.sub('https?://\S+|www\.\S+', ' ', text)
  
  # удаляем специальные символы
  # оставляем цифры
  text = re.sub('[^a-zA-Z0-9 ]+', ' ', text)

  # возвращаем текст, очищенный от лишних пробелов
  # которые появились в результате обработки
  return re.sub('\s+', ' ', text)

def digits_clean(text):
  ### удаляет все символы кроме букв ###

  # удаляем из текста ссылки 
  text = re.sub('https?://\S+|www\.\S+', ' ', text)
  
  # удаляем специальные символы
  # удаляем цифры
  text = re.sub('[^a-zA-Z ]+', ' ', text)

  # возвращаем текст, очищенный от лишних пробелов
  # которые появились в результате обработки
  return re.sub('\s+', ' ', text)

In [11]:
cleaned, full_cleaned, digits_cleaned = clean(test), full_clean(test), digits_clean(test)

print(f'Пример очистки текста\n\n{Fore.RED}Тестовый образец:{Style.RESET_ALL}\n"{test}"\n\n \
{Fore.RED}* Образец, очищенный с помощью функции clean (сохраняет знаки препинания и цифры){Style.RESET_ALL}\n"{cleaned}"\n\n \
{Fore.RED}* Образец, очищенный с помощью функции full_clean (очищает текст от знаков препинания){Style.RESET_ALL}\n"{full_cleaned}"\n\n \
{Fore.RED}* Образец, очищенный с помощью функции digits_clean (удаляет все символы кроме букв){Style.RESET_ALL}\n"{digits_cleaned}"')

Пример очистки текста

Тестовый образец:
"https://opentextbc.ca/physicalgeology2ed/
Physical Geology - 2nd Edition
Physical Geology - 2nd Edition
Steven Earle
Karla Panchuk
BCcampus
Victoria, B.C.
Icon for the Creative Commons Attribution 4.0 International License

Physical Geology - 2nd Edition by Steven Earle is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted.

© 2019 Steven Earle

The CC licence permits you to retain, reuse, copy, redistribute, and revise this book—in whole or in part—for free providing the author is attributed as follows:

Physical Geology – 2nd Edition by Steven Earle is used under a Creative Commons Attribution 4.0 International Licence.
If you redistribute all or part of this book, it is recommended the following statement be added to the copyright page so readers can access the original book at no cost:

Download for free from the B.C. Open Collection"

 * Образец, очищенный с помощью функции clean (сохраняе

## Токенизация

Рассмотрим несколько способов токенизации и создадим для каждого отдельную функцию.

In [12]:
# простая токенизация

def simple_tokenizer(text):
  # принимает на вход текст
  # приводит текст к нижнем регистру с помощью lower()
  # "рубит" текст на элементы, разделенные пробелом (слова)
  return text.lower().split(' ')

# токенизация методами NLTK

def nltk_tokenizer(text):
  # применяет метод word_tokenize из nltk к тексту
  return word_tokenize(text)

# токенизация с помощью gensim

def gensim_tokenizer(text, deacc):
  # применяет метод simple_preprocess из gensim к тексту
  # encode: кодировка текст
  # deacc удаляет знаки препинания, принимает значения True / False
  return simple_preprocess(str(text).encode('utf-8'), deacc)

In [13]:
tokenized, nltk_tokenized, gensim_tokenized = simple_tokenizer(cleaned), nltk_tokenizer(cleaned), gensim_tokenizer(cleaned,True)

print(f'Пример токенизации текста\n\n{Fore.RED}Тестовый образец:{Style.RESET_ALL}\n"{cleaned}"\n\n \
{Fore.RED}* Образец, токенизированный с помощью функции simple_tokenizer {Style.RESET_ALL}\n"{tokenized}"\n\n \
{Fore.RED}* Образец, токенизированный с помощью функции nltk_tokenizer {Style.RESET_ALL}\n"{nltk_tokenized}"\n\n \
{Fore.RED}* Образец, токенизированный с помощью функции gensim_tokenizer {Style.RESET_ALL}\n"{gensim_tokenized}"')

Пример токенизации текста

Тестовый образец:
" Physical Geology - 2nd Edition Physical Geology - 2nd Edition Steven Earle Karla Panchuk BCcampus Victoria, B.C. Icon for the Creative Commons Attribution 4.0 International License Physical Geology - 2nd Edition by Steven Earle is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted. 2019 Steven Earle The CC licence permits you to retain, reuse, copy, redistribute, and revise this book—in whole or in part—for free providing the author is attributed as follows: Physical Geology 2nd Edition by Steven Earle is used under a Creative Commons Attribution 4.0 International Licence. If you redistribute all or part of this book, it is recommended the following statement be added to the copyright page so readers can access the original book at no cost: Download for free from the B.C. Open Collection"

 * Образец, токенизированный с помощью функции simple_tokenizer 
"['', 'physical', 'geology', '-', '2

## Удаление стоп-слов

Попробуем три опции: 

- загрузка стоп-слов, которые предоставляет NLTK
- загрузка стоп-слов, которые предоставляет Spacy
- загрузка стоп-слов, которые предоставляет Gensim

In [14]:
def stopwords_nltk(text):
  # принимает токенизированный текст
  # сохраняем список слов в переменную
  sw = stopwords.words('english')

  # чтобы удалить стоп-слова, "пробежимся" по токенизированному списку
  # если слово отсутсвует в списке стоп-слов, сохраним его в новый список tokenized_no_sw
  return [x for x in text if not x in sw]

def stopwords_spacy(text):
  # загрузка spacy для английского языка
  en = spacy.load('en_core_web_sm')
  # загрузка стоп-слов
  sw = en.Defaults.stop_words

  return [x for x in text if not x in sw]

def stopwords_gensim(text):
  # загрузка стоп-слов из gensim
  sw = STOPWORDS
  
  return [x for x in text if not x in sw]

In [15]:
no_sw_nltk, no_sw_spacy, no_sw_gensim = stopwords_nltk(gensim_tokenized), stopwords_spacy(gensim_tokenized), stopwords_gensim(gensim_tokenized)

print(f'Пример очистки текста от стоп-слов\n\n{Fore.RED}Тестовый образец:{Style.RESET_ALL}\n"{gensim_tokenized}"\n\n \
{Fore.RED}* Образец, очищенный от стоп-слов с помощью функции NLTK {Style.RESET_ALL}\n"{no_sw_nltk}"\n\n \
{Fore.RED}* Образец, очищенный от стоп-слов с помощью функции Spacy {Style.RESET_ALL}\n"{no_sw_spacy}"\n\n \
{Fore.RED}* Образец, очищенный от стоп-слов с помощью функции Gensim {Style.RESET_ALL}\n"{no_sw_gensim}"')

Пример очистки текста от стоп-слов

Тестовый образец:
"['physical', 'geology', 'nd', 'edition', 'physical', 'geology', 'nd', 'edition', 'steven', 'earle', 'karla', 'panchuk', 'bccampus', 'victoria', 'icon', 'for', 'the', 'creative', 'commons', 'attribution', 'international', 'license', 'physical', 'geology', 'nd', 'edition', 'by', 'steven', 'earle', 'is', 'licensed', 'under', 'creative', 'commons', 'attribution', 'international', 'license', 'except', 'where', 'otherwise', 'noted', 'steven', 'earle', 'the', 'cc', 'licence', 'permits', 'you', 'to', 'retain', 'reuse', 'copy', 'redistribute', 'and', 'revise', 'this', 'book', 'in', 'whole', 'or', 'in', 'part', 'for', 'free', 'providing', 'the', 'author', 'is', 'attributed', 'as', 'follows', 'physical', 'geology', 'nd', 'edition', 'by', 'steven', 'earle', 'is', 'used', 'under', 'creative', 'commons', 'attribution', 'international', 'licence', 'if', 'you', 'redistribute', 'all', 'or', 'part', 'of', 'this', 'book', 'it', 'is', 'recommended', '

## Би- и триграммные модели Gensim

In [22]:
# Используем Phrases из gensim
# на вход модель принимает токенизированные данные
# чем выше min_count и threshold, тем меньше би- или триграм будет устанавливать модель
bigram = Phrases(no_sw_gensim, min_count=5, threshold=100)
trigram = Phrases(bigram[no_sw_gensim], threshold=100)  

bi_model = Phraser(bigram)
tri_model = Phraser(trigram)

print(f"{Fore.RED}Образец выдач биграммной модели{Style.RESET_ALL}\n", bi_model[no_sw_gensim])
print(f"{Fore.RED}Образец выдач триграммной модели{Style.RESET_ALL}\n", tri_model[no_sw_gensim])

Образец работы биграммной модели
 ['physical', 'geology', 'nd', 'edition', 'physical', 'geology', 'nd', 'edition', 'steven', 'earle', 'karla', 'panchuk', 'bccampus', 'victoria', 'icon', 'creative', 'commons', 'attribution', 'international', 'license', 'physical', 'geology', 'nd', 'edition', 'steven', 'earle', 'licensed', 'creative', 'commons', 'attribution', 'international', 'license', 'noted', 'steven', 'earle', 'cc', 'licence', 'permits', 'retain', 'reuse', 'copy', 'redistribute', 'revise', 'book', 'free', 'providing', 'author', 'attributed', 'follows', 'physical', 'geology', 'nd', 'edition', 'steven', 'earle', 'creative', 'commons', 'attribution', 'international', 'licence', 'redistribute', 'book', 'recommended', 'following', 'statement', 'added', 'copyright', 'page', 'readers', 'access', 'original', 'book', 'cost', 'download', 'free', 'open', 'collection']
Образец работы триграммной модели
 ['physical', 'geology', 'nd', 'edition', 'physical', 'geology', 'nd', 'edition', 'steven', '

# Сборка модели

In [96]:
def text_processing(text, cleaner, tokenizer, deacc, stopwords, split_volume):
  # полная предобработка текста
  # text - обучающие данные (str)
  # cleaner - тип очистки данных (function)
  # tokenizer - тип токенизатора (func)
  # deacc - для gensim_tokenizer, определить, удалять или нет знаки препинания (Bool)
  # stopwords - тип стоп-слов для удаления
  # split_volume - на какие отрывки будем делить текст для обработки (с помощью Numpy)
  text = cleaner(text)
  text = tokenizer(text, deacc) if tokenizer == gensim_tokenizer else tokenizer(text)
  text = stopwords(text)
  return np.array_split(text, split_volume)

# проводим полную предобработку сырых данных text
new_text = text_processing(text, full_clean, gensim_tokenizer, True, stopwords_gensim, 50)

print(f"{Fore.RED}Образец элементов обработанных данных:{Style.RESET_ALL}\n", new_text[0][:10])

# обучаем биграмную модель на наших данных
bigram_model = Phraser(Phrases(new_text, min_count=5, threshold=100))

# применяем модель
bigrams = [bigram_model[text] for text in new_text]

print(f"{Fore.RED}\n\nОбразец биграмм:{Style.RESET_ALL}\n", bigrams[0][:10])

Образец элементов обработанных данных:
 ['source' 'text' 'text' 'non' 'commercial' 'educational' 'purposes'
 'introduction' 'geology' 'free']


Образец биграмм:
 ['source', 'text', 'text', 'non_commercial', 'educational', 'purposes', 'introduction', 'geology', 'free', 'textbook']


# Term Frequency

In [100]:
# создание биграммного словаря с помощью gensim.corpora
id2word = Dictionary(bigrams)

# TF: подсчет частоты встречаемости элементов в корпусе
corpus = [id2word.doc2bow(text) for text in bigrams]
tf = [[(id2word[id], freq) for id, freq in cp] for cp in corpus]

print(f"{Fore.RED}Образец элементов словаря:{Style.RESET_ALL}\n", id2word[0], id2word[1], id2word[2])
print(f"{Fore.RED}Образец подсчета Term Frequency:{Style.RESET_ALL}\n", tf[0][:3])

Образец элементов словаря:
 able abraham absolute
Образец подсчета Term Frequency:
 [('able', 2), ('abraham', 1), ('absolute', 1)]


# Визуализация результата

In [135]:
# сортировка элементов, 
# добавляем все, что подсчитали, в общий список
to_sort  = []
for elem in tf:
  for x in elem:
    to_sort.append(x)

# создаем частотный словарь
freq_dict = dict()                                                                                       
for i in to_sort: 
    if i[0] not in freq_dict: 
        freq_dict[i[0]] = i[1] 
    else: 
        freq_dict[i[0]] += i[1] 

# создаем DataFrame с результатами работы 
df = pd.DataFrame(freq_dict.items(), columns=['item', 'frequency']).sort_values(by=['frequency'], ascending=False)
df.head()

,item,frequency
595,figure,2674
1394,rock,2008
1708,water,1613
1395,rocks,1418
464,earth,1214


In [136]:
# сохраянем в Excel

df.to_excel("output.xlsx")  